# I3D

In [1]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm
import pandas as pd

In [2]:
import json
from collections import Counter

In [ ]:
# from google.colab import drive

# # Mount Google Drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Load the metadata
top_100_path = './DataSet/gloss_counts_top_100.csv'
SAVE_PATH = "./DataSet/I3D 100"

# Assuming the uploaded CSV file is named 'top_100_classes.csv'
df = pd.read_csv(top_100_path)

# Extract the 'Gloss' column for the top 100 classes
top_100_classes = df['Gloss'].tolist()


In [ ]:
import numpy as np
import os
import psutil
import gc
from tqdm import tqdm
from tensorflow.keras.utils import to_categorical

# Function to check memory usage
def check_memory():
    memory = psutil.virtual_memory()
    print(f"Memory usage: {memory.percent}% of {memory.total / (1024 ** 3):.2f} GB")

# Initialize variables
sequences = []
labels = []
batch_size = 5  # Reduced batch size to prevent high memory usage

# Start processing the videos
for action in tqdm(top_100_classes, desc="Loading data for top 100 classes"):
    action_dir = os.path.join(SAVE_PATH, action)

    # Check if the directory exists for the action class
    if os.path.exists(action_dir):
        batch_sequences = []  # Temporary storage for the current batch
        batch_labels = []     # Temporary storage for the current batch

        for video_file in os.listdir(action_dir):
            if video_file.endswith('_frames.npy'):
                # Load the processed video frames
                video_path = os.path.join(action_dir, video_file)
                frames = np.load(video_path)

                # Append frames and labels to the current batch
                batch_sequences.append(frames)
                batch_labels.append(action)

                # If batch is full, process it
                if len(batch_sequences) == batch_size:
                    sequences.extend(batch_sequences)
                    labels.extend(batch_labels)

                    # Clear the current batch
                    batch_sequences = []
                    batch_labels = []

                    # Check memory usage after processing each batch
                    check_memory()
                    gc.collect()  # Explicitly clear memory

        # Process any remaining videos in the batch
        if batch_sequences:
            sequences.extend(batch_sequences)
            labels.extend(batch_labels)

            # Check memory usage after final batch
            check_memory()
            gc.collect()  # Explicitly clear memory

    else:
        print(f"Missing directory for class: {action}")

# Convert sequences and labels to NumPy arrays
X = np.array(sequences)
y = [top_100_classes.index(label) for label in labels]
y = to_categorical(y, num_classes=len(top_100_classes))  # One-hot encode the labels

# Print shapes to verify
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")


In [ ]:
# Save the data as X_i3d_100.npy and y_i3d_100.npy
np.save('./Dataset/final_Dataset/X_i3d_100.npy', X)
np.save('./Dataset/final_Dataset/y_i3d_100.npy', y)
